In [39]:
import os
import pandas as pd
import pyarrow.parquet as pq
from sqlalchemy import create_engine

In [40]:
# Caminho do diretório onde as pastas de arquivos Parquet estão armazenados
parquet_directory_path = 'gold/'

In [41]:
# Dados de conexão com o PostgreSQL
user = 'postgres'
password = 'root'
host = 'localhost'
port = '5432'
database = 'postgres'

In [42]:
# Criar a string de conexão com PostgreSQL
connection_string = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'

In [43]:
# Criar o motor de conexão
engine = create_engine(connection_string)

In [44]:

def carregar_arquivos_parquet(parquet_directory_path, engine):
    # Listar todos os diretórios na pasta gold/
    diretorios = [d for d in os.listdir(parquet_directory_path) if os.path.isdir(os.path.join(parquet_directory_path, d))]

    if not diretorios:
        print(f'Nenhuma subpasta encontrada em {parquet_directory_path}.')
        return

    for diretorio in diretorios:
        diretorio_path = os.path.join(parquet_directory_path, diretorio)
        arquivos = [f for f in os.listdir(diretorio_path) if f.endswith('.parquet')]

        if not arquivos:
            print(f'Nenhum arquivo Parquet encontrado na pasta {diretorio_path}.')
            continue

        for arquivo in arquivos:
            parquet_file_path = os.path.join(diretorio_path, arquivo)
            
            try:
                # Ler o arquivo Parquet
                df = pd.read_parquet(parquet_file_path, engine='pyarrow')
                print(f"Carregando dados de {arquivo}...")
                
                # Converter colunas de bytes para string 
                for col in df.select_dtypes(include=['object']).columns:
                    df[col] = df[col].astype(str)
                
                # Nome da tabela no PostgreSQL
                table_name = diretorio  # Nome da subpasta como nome da tabela
                
                try:
                    df.to_sql(table_name, engine, if_exists='replace', index=False)
                    print(f"Dados de {arquivo} carregados com sucesso na tabela {table_name}!")
                except Exception as e:
                    print(f"Erro ao carregar os dados de {arquivo}: {e}")
            
            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {e}")

In [45]:
# Chama a função para carregar os arquivos Parquet
carregar_arquivos_parquet(parquet_directory_path, engine)

Carregando dados de dim_avaliacao.parquet...
Dados de dim_avaliacao.parquet carregados com sucesso na tabela dim_avaliacao!
Carregando dados de dim_cliente.parquet...
Dados de dim_cliente.parquet carregados com sucesso na tabela dim_cliente!
Carregando dados de dim_geolocalizacao.parquet...
Dados de dim_geolocalizacao.parquet carregados com sucesso na tabela dim_geolocalizacao!
Carregando dados de dim_pagamento.parquet...
Dados de dim_pagamento.parquet carregados com sucesso na tabela dim_pagamento!
Carregando dados de dim_produto.parquet...
Dados de dim_produto.parquet carregados com sucesso na tabela dim_produto!
Carregando dados de dim_vendedor.parquet...
Dados de dim_vendedor.parquet carregados com sucesso na tabela dim_vendedor!
Carregando dados de fato_pedidos.parquet...
Dados de fato_pedidos.parquet carregados com sucesso na tabela fato_pedidos!
